Dr. Probe Python Wrapper.ipynb contains multislice simulation code for LACBED and unfinished multislice template matching code

In [18]:
import os
import subprocess

#Change operating directory to where the CLT is. Does not work on my Linux (MD 2020-12-18)
os.chdir('/home/martial/Data/programs/DR_probe/Exe_files/')

In [2]:
#Change operating directory to where the CLT is
path_drprobe = '/home/martial/Data/programs/DR_probe/Exe_files/'

In [3]:
# Import necesary packages 
import numpy as np
import matplotlib.pyplot as plt
import drprobe as drp
import math
from itertools import product 
from time import time
from scipy import ndimage
%matplotlib qt

In [4]:
# Set up phase gratings
nx=540
ny=243
nz = 6

In [5]:
#Projection of new cell in desired zone axis
prj=0,1,0,1,0,3,6.35674,6.35674,0.895 #First 3 numbers=Projection in x-axis. Following 3=Projection in y-axis. Last 3=new supercell size
tla=0,0,0 #Tilting multiplication along x,y,z for discretisation purpose

In [30]:
#inputs
input_data_path = '../../Demo_data/4D-STEM_Diffraction/'
abs_input_data_path = '/home/martial/OneDrive/Program/Git_NB/Notebook_Lision/Demo_data/4D-STEM_Diffraction/'
output_data_path = '/home/martial/Temp/'

#Add cif files at this stage and loop on the creation of slices below

#Folders to be used to save the msa.prn files (config files) and msa simulated data
pigeonite010_dir=output_data_path +'pigeonite010'
augite010_dir=output_data_path+'augite010'
msadirectory=[pigeonite010_dir,augite010_dir]

#Folders to be used to save the msa files
#pigeonite010_savedir=output_data_path+'pigeonite010/'
#augite010_savedir=output_data_path+'augite010/'
#savedirectory=[pigeonite010_savedir,augite010_savedir]

#input used to separate the phases simulated
phase=['pigeonite','augite']
phase[0]

'pigeonite'

#Celslc to orient cif file in desired zone axis. Notice=This function never works because of the space sent to the CLT from the prj variable. It is automatically assigned by the wrapper.
drp.os.chdir('/home/martial/Data/programs/DR_probe/Exe_files/')
drp.commands.celslc(cel_file='../../Demo_data/4D-STEM_Diffraction/pigeonite.cif', # location of cel/cif file
                    slice_name=output_data_path+phase[0]+'/cel/', # target file names
                    ht=300,       # high tension
                    nx=512,        # number of sampling points along x
                    ny=512,        # number of sampling points along y
                    nz=6,        # number of sampling points along z (number of slices)
                    absorb=False,  # apply absorptive form factors
                    dwf=False,     # apply Debye-Waller factors
                    fl=False,
                    output=True,  # Command line output (prints executed command)
                    pot=False,      # Saves potentials
                    prj=prj,
                    tla=tla,
                    rti=True
                   )

In [32]:
cmd = path_drprobe +'celslc -cif ' + abs_input_data_path + phase[0] + '.cif ' + \
'-slc '+ output_data_path + phase[0]+'/cel/ ' + \
'-ht 300 ' +\
'-nx 512 ' +\
'-ny 512 ' +\
'-nz 6 ' +\
'-fl ' +\
'-prj '+str(prj[0])+','+str(prj[1])+','+str(prj[2])+','+str(prj[3])+','+str(prj[4])+','+str(prj[5])+','+str(prj[6])+','+str(prj[7])+','+str(prj[7])+' '\
'-tla '+str(tla[0])+','+str(tla[1])+','+str(tla[2])

subprocess.call(cmd, shell=True)

0

In [33]:
cmd = path_drprobe +'celslc -cif ' + abs_input_data_path + phase[1]+ '.cif ' + \
'-slc '+ output_data_path + phase[0]+'/cel/ ' + \
'-ht 300 ' +\
'-nx 512 ' +\
'-ny 512 ' +\
'-nz 6 ' +\
'-fl ' +\
'-prj '+str(prj[0])+','+str(prj[1])+','+str(prj[2])+','+str(prj[3])+','+str(prj[4])+','+str(prj[5])+','+str(prj[6])+','+str(prj[7])+','+str(prj[7])+' '\
'-tla '+str(tla[0])+','+str(tla[1])+','+str(tla[2])

subprocess.call(cmd, shell=True)

0

In [34]:
#Set thickness list
thickness_list=np.arange(50,151,5)
for i,th in enumerate(thickness_list):
    slc_list=thickness_list
    slc_list[i]=math.floor(thickness_list[i]/(0.895/6))
thickness_list=np.arange(50,151,5)
thickness_list

array([ 50,  55,  60,  65,  70,  75,  80,  85,  90,  95, 100, 105, 110,
       115, 120, 125, 130, 135, 140, 145, 150])

In [35]:
#List of tilting angle
tilt_list=np.linspace(-1,1,11,dtype='float32')
tilt_list=list(product(tilt_list,repeat=2))
for tilt in tilt_list:
    print(tilt[0],tilt[1])

-1.0 -1.0
-1.0 -0.8
-1.0 -0.6
-1.0 -0.4
-1.0 -0.2
-1.0 0.0
-1.0 0.2
-1.0 0.4
-1.0 0.6
-1.0 0.8
-1.0 1.0
-0.8 -1.0
-0.8 -0.8
-0.8 -0.6
-0.8 -0.4
-0.8 -0.2
-0.8 0.0
-0.8 0.2
-0.8 0.4
-0.8 0.6
-0.8 0.8
-0.8 1.0
-0.6 -1.0
-0.6 -0.8
-0.6 -0.6
-0.6 -0.4
-0.6 -0.2
-0.6 0.0
-0.6 0.2
-0.6 0.4
-0.6 0.6
-0.6 0.8
-0.6 1.0
-0.4 -1.0
-0.4 -0.8
-0.4 -0.6
-0.4 -0.4
-0.4 -0.2
-0.4 0.0
-0.4 0.2
-0.4 0.4
-0.4 0.6
-0.4 0.8
-0.4 1.0
-0.2 -1.0
-0.2 -0.8
-0.2 -0.6
-0.2 -0.4
-0.2 -0.2
-0.2 0.0
-0.2 0.2
-0.2 0.4
-0.2 0.6
-0.2 0.8
-0.2 1.0
0.0 -1.0
0.0 -0.8
0.0 -0.6
0.0 -0.4
0.0 -0.2
0.0 0.0
0.0 0.2
0.0 0.4
0.0 0.6
0.0 0.8
0.0 1.0
0.2 -1.0
0.2 -0.8
0.2 -0.6
0.2 -0.4
0.2 -0.2
0.2 0.0
0.2 0.2
0.2 0.4
0.2 0.6
0.2 0.8
0.2 1.0
0.4 -1.0
0.4 -0.8
0.4 -0.6
0.4 -0.4
0.4 -0.2
0.4 0.0
0.4 0.2
0.4 0.4
0.4 0.6
0.4 0.8
0.4 1.0
0.6 -1.0
0.6 -0.8
0.6 -0.6
0.6 -0.4
0.6 -0.2
0.6 0.0
0.6 0.2
0.6 0.4
0.6 0.6
0.6 0.8
0.6 1.0
0.8 -1.0
0.8 -0.8
0.8 -0.6
0.8 -0.4
0.8 -0.2
0.8 0.0
0.8 0.2
0.8 0.4
0.8 0.6
0.8 0.8
0.8 1.0
1.0 -1.0
1.0 -0

In [36]:
# Create Parameter file for multislice
msa = drp.msaprm.MsaPrm()

In [38]:
# Setup (fundamental) MSA parameters
#for savedir,slcdir in zip(savedirectory,msadirectory):
for phase_def in phase:
    for slices,th in zip(slc_list,thickness_list):
        for tilt in tilt_list:
            msa.conv_semi_angle=3.0 #Convergence beam semi-angle. NOTE: The smoothing somehow can only be written manually into the msa.prm output
            msa.inner_radius_ann_det=0
            msa.outer_radius_ann_det=100
            msa.detector=0,'0'
            msa.wavelength = 0.001969  # wavelength (in nm) for 300 keV electrons
            msa.source_radius=0.4 #HWHM source profile radius
            msa.focus_spread=3
            msa.focus_spread_kernel_hw=2
            msa.focus_spread_kernel_size=7
            msa.number_of_aber=0
            msa.tilt_x=tilt[0] #Tilting angle in x-axis
            msa.tilt_y=tilt[1] #Tilting angle in y-axis
            msa.h_scan_offset=0
            msa.v_scan_offset=0
            msa.h_scan_frame_size=1
            msa.v_scan_frame_size=1
            msa.scan_frame_rot=0
            msa.scan_columns=1
            msa.scan_rows=1
            msa.spat_coherence_flag=1
            msa.super_cell_x=1
            msa.super_cell_y=1
            msa.slice_files =output_data_path+phase_def+'/cel/' # location of phase gratings
            msa.number_of_slices =6 # Number of slices in phase gratings
            msa.number_frozen_lattice=50
            msa.min_num_frozen=1
            msa.det_readout_period =0 # Readout at max thickness only
            msa.tot_number_of_slices=slices # indicate thickness
            msa.save_msa_prm(output_data_path+phase_def+'/msa/msa_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.prm',output=True)

Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x-1.0,y0.8.prm'!
Parameters succ

Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.6,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.8,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_50nm_x0.8,y-0.8.prm'!
Parameters successfully sa

Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_55nm_x0.0,y1.0.prm'!
Parameters successfully sa

Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.8,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.8,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.8,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.8,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_60nm_x-0.6,y0.0.prm'!
Parameters succ

Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x-0.2,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x-0.2,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x-0.2,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x-0.2,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x-0.2,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x0.0,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x0.0,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x0.0,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x0.0,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/pigeonite/msa/msa_65nm_x0.0,y-0.2.prm'!
Parameters successfu

...

Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x0.8,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_130nm_x1.0,y0.6.prm'!
Parameters successfully saved to file '/home/

Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.2,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.2,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_135nm_x0.4,y0.4.prm'!
Parameters successfully saved to file '/home/

Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.4,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.4,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.4,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.4,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y-0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y-0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_140nm_x-0.2,y0.0.prm'!
Parameters successfully saved to fi

Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.8,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.6,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.6,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.6,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_145nm_x-0.6,y-0.4.prm'!
Parameters successfully saved to fil

Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y0.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y0.2.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y0.4.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.8,y1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.6,y-1.0.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.6,y-0.8.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.6,y-0.6.prm'!
Parameters successfully saved to file '/home/martial/Temp/augite/msa/msa_150nm_x-0.6,y-0.4.prm'!
Parameters successfully saved to fil

#Execute library of multislice
#The wrapper does not work on linux, it seems it can't access the exe folder
tic=time()
for phase_def in phase:
    for th in thickness_list:
        for tilt in tilt_list:
            drp.commands.msa(prm_file=output_data_path+phase_def+'/msa/msa_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.prm',\
                             output_file=output_data_path+phase_def+'/img/img_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.dat',\
                             padif=True,output=True)
#'Simulation time: %.2fs'%(time()-tic)

In [ ]:
#Execute library of multislice
tic=time()
for phase_def in phase:
    for th in thickness_list:
        for tilt in tilt_list:
            cmd = path_drprobe +'msa -prm '+output_data_path+phase_def+'/msa/msa_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[0])+'.prm -out '+output_data_path+phase_def+'/img/img_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'.dat', 'padif=True,output=True'
            subprocess.call(cmd, shell=True)
'Simulation time: %.2fs'%(time()-tic)

# Display potentials
slc_1 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_001.pot', dtype='complex64').reshape((nx, ny))
slc_2 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_002.pot', dtype='complex64').reshape((nx, ny))
slc_3 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_003.pot', dtype='complex64').reshape((nx, ny))
slc_4 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_004.pot', dtype='complex64').reshape((nx, ny))
slc_5 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_005.pot', dtype='complex64').reshape((nx, ny))
slc_6 = np.fromfile('C:/Users/cqred/Documents/slc/pigeonite2_006.pot', dtype='complex64').reshape((nx, ny))

fig, axs = plt.subplots(1, 6, figsize=(25, 15))
axs[0].imshow(np.real(slc_1))
axs[1].imshow(np.real(slc_2))
axs[2].imshow(np.real(slc_3))
axs[3].imshow(np.real(slc_4))
axs[4].imshow(np.real(slc_5))
axs[5].imshow(np.real(slc_6))
axs[0].set_title('Slice 1')
axs[1].set_title('Slice 2')
axs[2].set_title('Slice 3')
axs[3].set_title('Slice 4')
axs[4].set_title('Slice 5')
axs[5].set_title('Slice 6')

In [ ]:
# Import necesary packages 
import numpy as np
import matplotlib.pyplot as plt
import drprobe as drp
import math
from itertools import product 
from time import time
from scipy import ndimage
%matplotlib qt

In [ ]:
# Import necesary packages 
import numpy as np
import matplotlib.pyplot as plt
import drprobe as drp
import math
from itertools import product 
from time import time
from scipy import ndimage
%matplotlib qt

In [ ]:
#function to crop np array
def crop_center(img,cropx,cropy):
    y,x = img.shape
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

In [ ]:
#Load .dat binary file of simulated STEM LACBED
img = np.fromfile('./pigeonite010_50nm_padif_tot_sl335.dat', dtype='float32')

In [ ]:
ny,nx= 498,540

In [ ]:
img=img.reshape((ny,nx))
img=np.append(img,img)
img=img.reshape((ny*2,nx))
img=np.append(img,img,axis=1)
img=np.flipud(img)

In [ ]:
img=crop_center(img,nx,ny)

In [ ]:
plt.imshow(img, cmap='gray',vmax=0.0005)

## Below is the code for Multislice template matching. However, it is not finished yet

In [ ]:
from pyxem.signals.indexation_results import TemplateMatchingResults

In [ ]:
def correlate(signal,phase,savedirectory,slc_list,thickness_list,tilt_list,
        n_largest=3,
        method="fast_correlation",
        *args,
        **kwargs,):  
                    matches = signal.map(
                                correlate_library,phase,savedirectory,slc_list,thickness_list,tilt_list,                                
                                n_largest=n_largest,
                                method=method,
                                inplace=False,
                                **kwargs,)

                    matching_results = TemplateMatchingResults(matches)
                    # matching_results = transfer_navigation_axes(matching_results, signal)

        return matching_results

In [ ]:
def correlate_library(signal,phase,savedirectory,slc_list,thickness_list,tilt_list, n_largest, method,rotation_angle):

    top_matches = np.empty((len(library), n_largest, 4), dtype="object")

    #if method == "zero_mean_normalized_correlation":
        #nb_pixels = image.shape[0] * image.shape[1]
        #average_image_intensity = np.average(image)
        #image_std = np.linalg.norm(image - average_image_intensity)
    
    rot_saved=np.empty((n_largest, 1),dtype=int)
    corr_saved=np.zeros((n_largest, 1))
    title_saved=np.empty((n_largest, 1),dtype='U16')

    for phase,savedir in zip(phase,savedirectory):
        for th,slc in zip(thickness_list,slc_list):
            for tilt in tilt_list:
                template=np.fromfile('C:/Users/cqred/Documents/'+phase+'/output/'+phase+'_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])+'_padif_tot_sl'+slc+'.dat',
                             dtype='float32')
                title=phase+'_'+str(th)+'nm_x'+str(tilt[0])+',y'+str(tilt[1])
                for angle in range(rotation_angle): #iterate through the rotations                                      
                        if method == "fast_correlation":
                            rotated_template = ndimage.rotate(template, angle, reshape=False, order=1)
                            pattern_norm=np.linalg.norm(rotated_template)
                            corr_local = fast_correlation(signal, rotated_template, pattern_norm)
                        if corr_local > np.min(corr_saved):
                            title_saved[np.argmin(corr_saved)]= title
                            rot_saved[np.argmin(corr_saved)]= angle
                            corr_saved[np.argmin(corr_saved)] = corr_local

                        combined_array = np.hstack((rot_saved,corr_saved,title_saved))
                        combined_array = combined_array[np.flip(combined_array[:, 3].argsort())] #sort array from smallest correlation value
                        top_matches[a, :, 0] = list(library)[a] #phase
                        top_matches[a, :, 1] = combined_array[:, 0] #thickness
                        top_matches[a, :, 2] = combined_array[:, 1] #tilting angle and axis
                        top_matches[a, :, 3] = combined_array[:, 2] #rotation angle
                        top_matches[a, :, 4] = combined_array[:, 3] #correlation

    return top_matches.reshape(-1, 4)

In [ ]:
dp = pxm.load('C:/Users/cqred/OneDrive/Documents/NTU/Y3S2/Project/DiffData/Diffraction SI.dm4')
dp.set_signal_type('electron_diffraction')
dp

In [ ]:
correlate(dp,phase,savedirectory,slc_list,thickness_list,tilt_list)

In [ ]:
#Template matching of variable 'data'
indexer = IndexationGenerator(data, diff_lib)
indexation_results = indexer.correlate(n_largest=3)